In [1]:
## imports
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib
import mne
import re
import os

from mne.preprocessing import (
    ICA, 
    create_ecg_epochs, 
    corrmap, 
    create_eog_epochs
)

# mne graphics config
# from PySide6 import QtCore, QtGui, QtWidgets, __version__ 

rawa = None
rawb = None

import datetime
print('IMPORTED MODULES today:', datetime.datetime.now())

# Removed cached function import
import sys
try:
    del sys.modules['rename_events']
except KeyError:
    pass

from rename_events import relabel_events


# setup

# SESSION AND POSNER CODES
session_id='m_09_02'
posner_id ='pos2a'
posner_id_b ='pos2b'

IMPORTED MODULES today: 2023-07-17 22:01:04.443936


# Posner A

In [2]:
# Relabel stim events and load raw data
raw_a = relabel_events(session_id, posner_id)

Data file:  m_09_02/m_09_02_pos2a.vhdr


/Users/babe/src/gla/dissertation/go/ica_notebooks/rename_events.py:50: RuntimeWarning: No coordinate information found for channels ['ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  raw = mne.io.read_raw_brainvision(vhdr_file, preload=True, verbose=0)


Used Annotations descriptions: ['New Segment/', 'Stimulus/S 10', 'Stimulus/S 11', 'Stimulus/S 30', 'Stimulus/S 40', 'Stimulus/S 50', 'Stimulus/S 51', 'Stimulus/S 60', 'Stimulus/S 70', 'Stimulus/S 80', 'Stimulus/S101', 'Stimulus/S109']
Found '2a' in file: m_09_02/09_2a_posner_task_2023-05-26_18h24.07.093/09_2a_posner_task_2023-05-26_18h24.07.093.csv
Found '2b' in file: m_09_02/09_2b_posner_task_2023-05-26_19h28.57.453/09_2b_posner_task_2023-05-26_19h28.57.453.csv
********** 2a
Used Annotations descriptions: ['New Segment/', 'Stimulus/S 10', 'Stimulus/S 11', 'Stimulus/S 30', 'Stimulus/S 40', 'Stimulus/S 50', 'Stimulus/S 51', 'Stimulus/S 60', 'Stimulus/S 70', 'Stimulus/S 80', 'Stimulus/S101', 'Stimulus/S109']
Cropping annotations 2023-05-26 18:26:29.255275+00:00 - 2023-05-26 18:40:27.035275+00:00
  [0] Keeping  (2023-05-26 18:26:29.255275+00:00 - 2023-05-26 18:26:29.255275+00:00 -> 0.0 - 0.0)
  [1] Keeping  (2023-05-26 18:27:37.142275+00:00 - 2023-05-26 18:27:37.142275+00:00 -> 67.887 - 6

/Users/babe/src/gla/dissertation/go/ica_notebooks/rename_events.py:50: RuntimeWarning: Not setting position of 1 misc channel found in montage:
['ECG']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  raw = mne.io.read_raw_brainvision(vhdr_file, preload=True, verbose=0)


  [321] Keeping  (2023-05-26 18:37:23.524275+00:00 - 2023-05-26 18:37:23.524275+00:00 -> 654.269 - 654.269)
  [322] Keeping  (2023-05-26 18:37:23.624275+00:00 - 2023-05-26 18:37:23.624275+00:00 -> 654.369 - 654.369)
  [323] Keeping  (2023-05-26 18:37:25.934275+00:00 - 2023-05-26 18:37:25.934275+00:00 -> 656.679 - 656.679)
  [324] Keeping  (2023-05-26 18:37:27.434275+00:00 - 2023-05-26 18:37:27.434275+00:00 -> 658.179 - 658.179)
  [325] Keeping  (2023-05-26 18:37:30.384275+00:00 - 2023-05-26 18:37:30.384275+00:00 -> 661.129 - 661.129)
  [326] Keeping  (2023-05-26 18:37:30.484275+00:00 - 2023-05-26 18:37:30.484275+00:00 -> 661.229 - 661.229)
  [327] Keeping  (2023-05-26 18:37:32.434275+00:00 - 2023-05-26 18:37:32.434275+00:00 -> 663.179 - 663.179)
  [328] Keeping  (2023-05-26 18:37:33.934275+00:00 - 2023-05-26 18:37:33.934275+00:00 -> 664.679 - 664.679)
  [329] Keeping  (2023-05-26 18:37:37.204275+00:00 - 2023-05-26 18:37:37.204275+00:00 -> 667.949 - 667.949)
  [330] Keeping  (2023-05-26

In [3]:
event_map = {'1': 11, '10': 10, '101': 101, '109': 109, '11': 11, '117': 117, '20': 20, '30': 30, '40': 40, '5': 5, '50': 50, '51': 51, '52': 52, '60': 60, '70': 70, '80': 80, '99999': 99999}

# get events and times
events, event_id = mne.events_from_annotations(raw_a, event_map)

e_a = events
# all rows with fixation (code 40)
e_40_a = e_a[e_a[:, 2] == 40]
# all rows with trial end (code 101)
e_101_a = e_a[e_a[:,2]==101]

a_tmin= e_40_a[0][0]/1000
a_tmax= e_101_a[-1][0]/1000

# Crop and make a copy and filter
filt_raw_a = raw_a.crop(tmin=a_tmin, tmax=a_tmax).copy().filter(l_freq=1.0, h_freq=40)

epochs_a = mne.Epochs(filt_raw_a, 
                    e_a, 
                    event_id=None, 
                    tmin=-0.2, 
                    tmax=0.5, 
                    preload=True, 
                    baseline=(None, 0))

Used Annotations descriptions: ['10', '101', '109', '11', '30', '40', '50', '51', '52', '60', '70', '80', '99999']
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (3.301 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Not setting metadata
412 matching events found
Setting baseline interval to [-0.2, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 412 events and 701 original time points ...
106 bad epochs dropped


[Parallel(n_jobs=1)]: Done  63 out of  63 | elapsed:    0.5s finished


In [50]:
raw_a.plot()

Using pyopengl with version 3.1.6


Channels marked as bad:
none
Channels marked as bad:
none


In [4]:
# Fitting ICA
ica = ICA(n_components=len(filt_raw_a.ch_names)-1, max_iter="auto", random_state=999)
ica.fit(filt_raw_a)
ica

Fitting ICA to data using 63 channels (please be patient, this may take a while)
Selecting by number: 63 components
Fitting ICA took 42.3s.


Method,fastica
Fit,80 iterations on raw data (516739 samples)
ICA components,63
Available PCA components,63
Channel types,eeg
ICA components marked for exclusion,—


In [6]:
%matplotlib qt
ica.plot_components()

[<MNEFigure size 975x967 with 20 Axes>,
 <MNEFigure size 975x967 with 20 Axes>,
 <MNEFigure size 975x967 with 20 Axes>,
 <MNEFigure size 585x260 with 3 Axes>]

In [ ]:
raw_a.load_data()
ica.plot_sources(raw_a)

Creating RawArray with float64 data, n_channels=63, n_times=516739
    Range : 118526 ... 635264 =    118.526 ...   635.264 secs
Ready.
Using qt as 2D backend.
Using pyopengl with version 3.1.6


    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
258 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
258 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
258 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
258 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
258 matching events found
No baseline correction applied
0 projection items activated


In [7]:
print(ica.exclude)

[43, 0, 1, 16, 15, 17, 22, 21, 26, 24, 31, 25, 14, 42, 4, 11, 55]


In [8]:
reconstr_raw = raw_a.copy()
# Apply ica to copy
ica.apply(reconstr_raw)
# Plot
raw_a.plot(title="Raw data")
reconstr_raw.plot(title="ICA applied")

Applying ICA to Raw instance
    Transforming to ICA space (63 components)
    Zeroing out 17 ICA components
    Projecting back using 63 PCA components
Using qt as 2D backend.
Using pyopengl with version 3.1.6
Using pyopengl with version 3.1.6


Channels marked as bad:
none
Channels marked as bad:
none


In [9]:
# Save file
file_name = session_id+'_'+posner_id+'.vhdr'
output_dir='cleaned'

print(file_name)
cur_dir = os.getcwd()
session_dir = os.path.join(cur_dir, output_dir, session_id)
print(session_dir)

if not os.path.exists(session_dir):
   os.makedirs(session_dir)

print("\n**** Saving file to: ", session_dir+'/'+file_name) 
mne.export.export_raw(session_dir+'/'+file_name, reconstr_raw, verbose=1)

m_09_02_pos2a.vhdr
/Users/babe/src/gla/dissertation/go/ica_notebooks/cleaned/m_09_02

**** Saving file to:  /Users/babe/src/gla/dissertation/go/ica_notebooks/cleaned/m_09_02/m_09_02_pos2a.vhdr


/Applications/MNE-Python/.mne-python/lib/python3.10/site-packages/pybv/_export.py:61: RuntimeWarning: Encountered data in 'short' format. Converting to float32.
  warn(
/Applications/MNE-Python/.mne-python/lib/python3.10/site-packages/pybv/io.py:690: UserWarning: Encountered unsupported non-voltage units: n/a
Note that the BrainVision format specification supports only µV.
  warn(msg)


In [10]:
!tree {session_dir}

/Users/babe/src/gla/dissertation/go/ica_notebooks/cleaned/m_09_02
├── m_09_02_pos2a.eeg
├── m_09_02_pos2a.vhdr
└── m_09_02_pos2a.vmrk

1 directory, 3 files


In [11]:
del raw_a
del reconstr_raw
del ica
del filt_raw_a

# Posner B

In [12]:
# Relabel stim events and load raw data
raw_b = relabel_events(session_id, posner_id_b)

Data file:  m_09_02/m_09_02_pos2b.vhdr


/Users/babe/src/gla/dissertation/go/ica_notebooks/rename_events.py:50: RuntimeWarning: No coordinate information found for channels ['ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  raw = mne.io.read_raw_brainvision(vhdr_file, preload=True, verbose=0)


Used Annotations descriptions: ['New Segment/', 'Stimulus/S 10', 'Stimulus/S 30', 'Stimulus/S 40', 'Stimulus/S 50', 'Stimulus/S 51', 'Stimulus/S 60', 'Stimulus/S 70', 'Stimulus/S 80', 'Stimulus/S101', 'Stimulus/S109']
Found '2a' in file: m_09_02/09_2a_posner_task_2023-05-26_18h24.07.093/09_2a_posner_task_2023-05-26_18h24.07.093.csv
Found '2b' in file: m_09_02/09_2b_posner_task_2023-05-26_19h28.57.453/09_2b_posner_task_2023-05-26_19h28.57.453.csv
********** 2b
Used Annotations descriptions: ['New Segment/', 'Stimulus/S 10', 'Stimulus/S 30', 'Stimulus/S 40', 'Stimulus/S 50', 'Stimulus/S 51', 'Stimulus/S 60', 'Stimulus/S 70', 'Stimulus/S 80', 'Stimulus/S101', 'Stimulus/S109']
Cropping annotations 2023-05-26 19:31:09.087401+00:00 - 2023-05-26 19:44:05.967401+00:00
  [0] Keeping  (2023-05-26 19:31:09.087401+00:00 - 2023-05-26 19:31:09.087401+00:00 -> 0.0 - 0.0)
  [1] Keeping  (2023-05-26 19:31:39.781401+00:00 - 2023-05-26 19:31:39.781401+00:00 -> 30.694 - 30.694)
  [2] Keeping  (2023-05-26 

/Users/babe/src/gla/dissertation/go/ica_notebooks/rename_events.py:50: RuntimeWarning: Not setting position of 1 misc channel found in montage:
['ECG']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  raw = mne.io.read_raw_brainvision(vhdr_file, preload=True, verbose=0)


In [13]:
event_map = {'1': 11, '10': 10, '101': 101, '109': 109, '11': 11, '117': 117, '20': 20, '30': 30, '40': 40, '5': 5, '50': 50, '51': 51, '52': 52, '60': 60, '70': 70, '80': 80, '99999': 99999}

# get events and times
events_b, event_id_b = mne.events_from_annotations(raw_b, event_map)

e_b = events_b
# all rows with fixation (code 40)
e_40_b = e_b[e_b[:, 2] == 40]
# all rows with trial end (code 101)
e_101_b = e_b[e_b[:,2]==101]

b_tmin= e_40_b[0][0]/1000
b_tmax= e_101_b[-1][0]/1000

# Crop and make a copy and filter
filt_raw_b = raw_b.crop(tmin=b_tmin, tmax=b_tmax).copy().filter(l_freq=1.0, h_freq=40)

epochs_b = mne.Epochs(filt_raw_b, 
                    e_b, 
                    event_id=None, 
                    tmin=-0.2, 
                    tmax=0.5, 
                    preload=True, 
                    baseline=(None, 0))

Used Annotations descriptions: ['10', '101', '109', '30', '40', '50', '51', '52', '60', '70', '80', '99999']
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (3.301 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Not setting metadata
411 matching events found
Setting baseline interval to [-0.2, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 411 events and 701 original time points ...
105 bad epochs dropped


[Parallel(n_jobs=1)]: Done  63 out of  63 | elapsed:    0.5s finished


In [5]:
raw_b.plot()

Using qt as 2D backend.
Using pyopengl with version 3.1.6


Channels marked as bad:
none


In [14]:
# Fitting ICA
ica_b = ICA(n_components=len(filt_raw_b.ch_names)-1, max_iter="auto", random_state=999)
ica_b.fit(filt_raw_b, verbose='DEBUG')
ica_b

Fitting ICA to data using 63 channels (please be patient, this may take a while)
Selecting by number: 63 components
Cropping annotations 2023-05-26 19:32:18.546401+00:00 - 2023-05-26 19:40:55.166401+00:00
  [0] Keeping  (2023-05-26 19:32:18.546401+00:00 - 2023-05-26 19:32:18.546401+00:00 -> 69.459 - 69.459)
  [1] Keeping  (2023-05-26 19:32:20.046401+00:00 - 2023-05-26 19:32:20.046401+00:00 -> 70.959 - 70.959)
  [2] Keeping  (2023-05-26 19:32:23.756401+00:00 - 2023-05-26 19:32:23.756401+00:00 -> 74.669 - 74.669)
  [3] Keeping  (2023-05-26 19:32:23.856401+00:00 - 2023-05-26 19:32:23.856401+00:00 -> 74.769 - 74.769)
  [4] Keeping  (2023-05-26 19:32:25.056401+00:00 - 2023-05-26 19:32:25.056401+00:00 -> 75.969 - 75.969)
  [5] Keeping  (2023-05-26 19:32:26.556401+00:00 - 2023-05-26 19:32:26.556401+00:00 -> 77.469 - 77.469)
  [6] Keeping  (2023-05-26 19:32:30.356401+00:00 - 2023-05-26 19:32:30.356401+00:00 -> 81.269 - 81.269)
  [7] Keeping  (2023-05-26 19:32:30.456401+00:00 - 2023-05-26 19:32

Method,fastica
Fit,94 iterations on raw data (516620 samples)
ICA components,63
Available PCA components,63
Channel types,eeg
ICA components marked for exclusion,—


In [18]:
ica_b.plot_components()

[<MNEFigure size 975x967 with 20 Axes>,
 <MNEFigure size 975x967 with 20 Axes>,
 <MNEFigure size 1950x1832 with 20 Axes>,
 <MNEFigure size 585x260 with 3 Axes>]

In [42]:
raw_b.load_data()
ica_b.plot_sources(raw_b)

Creating RawArray with float64 data, n_channels=63, n_times=516620
    Range : 69459 ... 586078 =     69.459 ...   586.078 secs
Ready.
Using pyopengl with version 3.1.6


In [19]:
ica_b.exclude

[0, 1, 5, 14, 18, 57, 48, 46, 35, 21, 54, 17, 24]

In [20]:
reconstr_raw_b = raw_b.copy()
# Apply ica to copy
ica_b.apply(reconstr_raw_b)
# Plot and compare
raw_b.plot(title="Raw data (posner b)")
reconstr_raw_b.plot(title="ICA applied (posner b)")

Applying ICA to Raw instance
    Transforming to ICA space (63 components)
    Zeroing out 13 ICA components
    Projecting back using 63 PCA components
Using pyopengl with version 3.1.6
Using pyopengl with version 3.1.6


Channels marked as bad:
none
Channels marked as bad:
none


In [21]:
# Save file
file_name_b = session_id+'_'+posner_id_b+'.vhdr'
output_dir_b='cleaned'
cur_dir = os.getcwd()
session_dir = os.path.join(cur_dir, output_dir_b, session_id)
if not os.path.exists(session_dir):
   os.makedirs(session_dir)

print("\n**** Saving file to: ", session_dir+'/'+file_name_b) 
mne.export.export_raw(session_dir+'/'+file_name_b, reconstr_raw_b, verbose=1, overwrite=True)


**** Saving file to:  /Users/babe/src/gla/dissertation/go/ica_notebooks/cleaned/m_09_02/m_09_02_pos2b.vhdr


/Applications/MNE-Python/.mne-python/lib/python3.10/site-packages/pybv/_export.py:61: RuntimeWarning: Encountered data in 'short' format. Converting to float32.
  warn(
/Applications/MNE-Python/.mne-python/lib/python3.10/site-packages/pybv/io.py:690: UserWarning: Encountered unsupported non-voltage units: n/a
Note that the BrainVision format specification supports only µV.
  warn(msg)


In [22]:
!tree {session_dir}

/Users/babe/src/gla/dissertation/go/ica_notebooks/cleaned/m_09_02
├── m_09_02_pos2a.eeg
├── m_09_02_pos2a.vhdr
├── m_09_02_pos2a.vmrk
├── m_09_02_pos2b.eeg
├── m_09_02_pos2b.vhdr
└── m_09_02_pos2b.vmrk

1 directory, 6 files


# ...

In [23]:
del raw_b
del reconstr_raw_b
del ica_b
del filt_raw_b

# Reload TEST

In [11]:
test_file = './cleaned/m_09_02/m_09_02_pos2b.vhdr'
raw_test = mne.io.read_raw(test_file)

Extracting parameters from ./cleaned/m_09_02/m_09_02_pos2b.vhdr...
Setting channel info structure...


In [14]:
raw_test.set_montage(montage)
raw_test.info.set_montage(montage)
filt_test = raw_test.copy().filter(1, 40)
ica_test = ICA(n_components=len(raw_test.ch_names)-1, max_iter="auto", random_state=999)
ica_test.fit(filt_test)
ica_test

/var/folders/2b/4tnqzv_x3x17pf4q1_n7m8h40000gn/T/ipykernel_36269/1861574689.py:1: RuntimeWarning: Fiducial point nasion not found, assuming identity unknown to head transformation
  raw_test.set_montage(montage)
/var/folders/2b/4tnqzv_x3x17pf4q1_n7m8h40000gn/T/ipykernel_36269/1861574689.py:1: RuntimeWarning: Not setting position of 1 misc channel found in montage:
['ECG']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  raw_test.set_montage(montage)
/var/folders/2b/4tnqzv_x3x17pf4q1_n7m8h40000gn/T/ipykernel_36269/1861574689.py:2: RuntimeWarning: Fiducial point nasion not found, assuming identity unknown to head transformation
  raw_test.info.set_montage(montage)
/var/folders/2b/4tnqzv_x3x17pf4q1_n7m8h40000gn/T/ipykernel_36269/1861574689.py:2: RuntimeWarning: Not setting position of 1 misc channel found in montage:
['ECG']
Consider setting the channel typ

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (3.301 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s


Fitting ICA to data using 63 channels (please be patient, this may take a while)


[Parallel(n_jobs=1)]: Done  63 out of  63 | elapsed:    0.7s finished


Selecting by number: 63 components
Fitting ICA took 34.7s.


/var/folders/2b/4tnqzv_x3x17pf4q1_n7m8h40000gn/T/ipykernel_36269/1861574689.py:5: RuntimeWarning: Using n_components=63 (resulting in n_components_=63) may lead to an unstable mixing matrix estimation because the ratio between the largest (31) and smallest (5.2e-15) variances is too large (> 1e6); consider setting n_components=0.999999 or an integer <= 62
  ica_test.fit(filt_test)


Method,fastica
Fit,57 iterations on raw data (516620 samples)
ICA components,63
Available PCA components,63
Channel types,eeg
ICA components marked for exclusion,—


In [ ]:
# raw_test.drop_channels('ECG')
montage = mne.channels.read_custom_montage('./BC-MR3-64.bvef')
# raw_test.drop_channels('ECG')
raw_test.load_data()
ica_test.plot_sources(raw_test)

Creating RawArray with float64 data, n_channels=63, n_times=516620
    Range : 0 ... 516619 =      0.000 ...   516.619 secs
Ready.
Using pyopengl with version 3.1.6


    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
258 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
258 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
258 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
258 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
258 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
258 matching events found
No baseline correction applied
0 projection items ac